In [15]:
import sys 
import argparse
sys.path.append('./project/')
sys.path.append(f'/users/Etu2/3701222/.local/lib/python3.7/site-packages')


import pytorch_lightning as pl
from vision_transformer_org import VisionTransformer
from lightning_modules import CIFAR10DataModule, LitClassifierModel, load_from_checkpoint

In [ ]:
!python ./project/main.py --fit --default_root_dir "/tmp/model" --data_path "/tmp/cifar10" --progress_bar_refresh_rate 25

In [19]:
args = argparse.Namespace(
    fit=True,
    default_root_dir="/tmp/model",
    data_path="/tmp/cifar10",
    gpus=0,
    learning_rate=1e-4,
    val_size=0.01,
    batch_size=16,
    image_size=32,
    num_workers=8,
    patch_size=4,
    emb_dim=64,
    mlp_dim=128,
    num_heads=4,
    num_layers=1,
    num_classes=10,
    attn_dropout_rate=0.0,
    dropout_rate=0.3,
    progress_bar_refresh_rate=25,
)

In [22]:
datamodule = CIFAR10DataModule(
        data_dir=args.data_path,
        image_size=args.image_size,
        batch_size=args.batch_size,
        num_workers=args.num_workers,
        val_size=args.val_size
)

vit_Backbone = VisionTransformer(
        image_size=(args.image_size, args.image_size),
        patch_size=(args.patch_size, args.patch_size),
        emb_dim=args.emb_dim,
        mlp_dim=args.mlp_dim,
        num_heads=args.num_heads,
        num_layers=args.num_layers,
        num_classes=args.num_classes,
        attn_dropout_rate=args.attn_dropout_rate,
        dropout_rate=args.dropout_rate,
)

checkpoint_callback = ModelCheckpoint(
        monitor='val_acc',
        filename='vit-{epoch:02d}-{val_loss:.2f}-{val_acc:.2f}',
        mode='max',
)
lr_monitor = LearningRateMonitor(logging_interval='step')


In [23]:
trainer = pl.Trainer.from_argparse_args(args, callbacks=[checkpoint_callback, lr_monitor])

GPU available: False, used: False
TPU available: None, using: 0 TPU cores


In [24]:
model = LitClassifierModel(vit_Backbone, **vars(args))

In [ ]:
trainer.fit(model, datamodule)

Files already downloaded and verified
Files already downloaded and verified



  | Name     | Type              | Params
-----------------------------------------------
0 | backbone | VisionTransformer | 41.6 K
-----------------------------------------------
41.6 K    Trainable params
0         Non-trainable params
41.6 K    Total params
0.166     Total estimated model params size (MB)
